In [4]:
import requests  # 서버에 http 프로토콜로 요청
from bs4 import BeautifulSoup as bs  # html 파싱
import time  # 5초간 휴식 할 때 사용
from datetime import datetime  # 오늘 연월일 추출
import pandas as pd  # 데이터 프레임 만듬

In [11]:
today = datetime.today()
yyyy = today.year
mm = today.month
print(yyyy, mm)

2023 9


In [18]:
year = list(range(2023, 2021, -1))
month = list(range(1, 13))
page = list(range(1, 10))
print(year)
print(month)
print(page)


[2023, 2022]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [109]:
result = []
for yy in year:
    for mon in month if yy != yyyy else range(1, mm):
        for p in page:
            print(f"{yy}년{mon}월{p}페이지 수집중", end="\r")
            url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller"
            payload = dict(categoryNumber='001', pageNumber=p, pageSize=120, type='month', saleYear=yy, saleMonth=mon)
            
            try:
                r = requests.get(url, params=payload) # yes24 서버에 url로 자료 요청
            except Exception as e:
                print(e)
            
            soup = bs(r.text, 'lxml')   # yes24 서버에서 받은 응답 값 텍스트를 beautifulsoup을 통해 html로 파싱
            book_list = soup.select("#yesBestList > li")
            for book in book_list:
                title_h = book.select_one(".gd_res").get_text()
                title_f = book.select_one(".gd_nameF").get_text() if book.select_one(".gd_nameF") != None else ""
                title_m = book.select_one(".gd_name").get_text() if book.select_one(".gd_name") != None else "제목없음"
                title_e = book.select_one(".gd_nameE").get_text() if book.select_one(".gd_nameE") != None else ""
                detail_link = "https://www.yes24.com" + book.select_one(".gd_name")["href"] if book.select_one(".gd_name") != None else "제목없음"
                author = [i.get_text() for i in book.select(".authPub.info_auth > a") if book.select(".authPub.info_auth > a") != None]
                publisher = book.select_one(".authPub.info_pub > a").get_text() if book.select_one(".authPub.info_pub > a") != None else ""
                pub_date = book.select_one(".authPub.info_date").get_text() if book.select_one(".authPub.info_date") != None else ""
                price = book.select_one(".yes_b").get_text() if book.select_one(".yes_b") != None else 0
                n_reviews = book.select_one(".txC_blue").get_text() if book.select_one(".txC_blue") != None else 0
                review_link = book.select_one(".rating_rvCount > a")['href'] if book.select_one(".rating_rvCount > a") != None else ""
                rating = book.select_one("span.rating_grade > .yes_b").get_text() if book.select_one("span.rating_grade > .yes_b") != None else 0
                tags = [i.get_text() for i in book.select(".info_row.info_tag > .tag > a") if book.select(".info_row.info_tag > .tag > a") != None]
                result.append([yy, mon, title_h, title_f, title_m, title_e, detail_link, author, publisher, pub_date, price, n_reviews,
                     review_link, rating, tags])
                time.sleep(5)

KeyboardInterrupt: 

In [101]:
len(result)

480

In [ ]:
result

In [108]:
df = pd.DataFrame(result, columns=['year', 'month', 'title_h', 'title_f', 'title_m', 'title_e', 'detail_link', 'author', 'publisher', 'pub_date', 'price', 'n_reviews',
                     'review_link', 'rating', 'tags'])
df

,year,month,title_h,title_f,title_m,title_e,detail_link,author,publisher,pub_date,price,n_reviews,review_link,rating,tags
0,2023,1,[도서],,만일 내가 인생을 다시 산다면,벌써 마흔이 된 당신에게 해 주고 싶은 말들 42,https://www.yes24.com/Product/Goods/115142458,[김혜남],메이븐,2022년 11월,"15,480",133,https://www.yes24.com/Product/Goods/115142458?...,9.6,"[#2023대구올해의책, #크레마클럽에있어요, #인생선배, #인생2막대비, #내인생..."
1,2023,1,[도서],,스카이 버스,명문 대학으로 직행하는 초등 공부 전략서,https://www.yes24.com/Product/Goods/116579452,[분당강쌤],다산에듀,2023년 01월,"16,200",244,https://www.yes24.com/Product/Goods/116579452?...,9.8,[#초등공부]
2,2023,1,[도서],,생에 감사해,,https://www.yes24.com/Product/Goods/116046489,[김혜자],수오서재,2022년 12월,"15,300",57,https://www.yes24.com/Product/Goods/116046489?...,9.6,"[#문학뉴스레터에소개된책, #감사하는삶, #삶을노래하다, #배우의삶, #연예인에세이]"
3,2023,1,[도서],,불편한 편의점,,https://www.yes24.com/Product/Goods/99308021,[김호연],나무옆의자,2021년 04월,"12,600",518,https://www.yes24.com/Product/Goods/99308021?R...,9.2,"[#2022포천시올해의책, #2022광양시올해의책, #크레마클럽에있어요, #흩날리는..."
4,2023,1,[도서],,불편한 편의점 2,,https://www.yes24.com/Product/Goods/111088149,[김호연],나무옆의자,2022년 08월,"12,600",199,https://www.yes24.com/Product/Goods/111088149?...,9.2,"[#크레마클럽에있어요, #가을에어울리는책, #이웃의풍경, #현실을그렸어요, #편의점에서]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2023,2,[도서],,8주간의 기적 근육의 부활,"망가진 몸, 저질체력을 완전히 날려버리는 짐승 트레이닝",https://www.yes24.com/Product/Goods/40212809,[조명기],청림Life,2017년 06월,"19,800",24,https://www.yes24.com/Product/Goods/40212809?R...,9.2,[]
476,2023,2,[도서],,2023 산업안전기사 실기[필답형+작업형]+무료동영상,,https://www.yes24.com/Product/Goods/117335554,[최윤정],구민사,2023년 02월,"37,800",6,https://www.yes24.com/Product/Goods/117335554?...,9.3,[]
477,2023,2,[도서],,오십에 읽는 논어,굽이치는 인생을 다잡아 주는 공자의 말,https://www.yes24.com/Product/Goods/104647964,[최종엽],유노북스,2021년 11월,"14,400",70,https://www.yes24.com/Product/Goods/104647964?...,9.1,[]
478,2023,2,[도서],,잘했고 잘하고 있고 잘 될 것이다 (스페셜 리미티드 에디션),,https://www.yes24.com/Product/Goods/101682634,[정영욱],부크럼,2022년 10월,"13,500",132,https://www.yes24.com/Product/Goods/101682634?...,9.8,[]


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   yy           480 non-null    int64 
 1   mon          480 non-null    int64 
 2   title_h      480 non-null    object
 3   title_f      480 non-null    object
 4   title_m      480 non-null    object
 5   title_e      480 non-null    object
 6   detail_link  480 non-null    object
 7   author       480 non-null    object
 8   publisher    480 non-null    object
 9   pub_date     480 non-null    object
 10  price        480 non-null    object
 11  n_reviews    480 non-null    object
 12  review_link  480 non-null    object
 13  rating       480 non-null    object
 14  tags         480 non-null    object
dtypes: int64(2), object(13)
memory usage: 56.4+ KB
